In [1]:
import json
import os
import pickle
import torch
import numpy as np

In [2]:
# Read torch 
def read_torch(fname):
    obj = torch.load(fname)
    return obj

In [27]:
models = ['arbo', '1nn', '1rand', 'knn', 'in_batch']
cands = read_torch(f'data/zeshel/top64_cands/{models[0]}.t7')
cands1 = read_torch(f'data/zeshel/top64_cands/{models[1]}.t7')

In [24]:
dataset = 'zeshel'
l = np.array([False]*(10000 if dataset == 'zeshel' else 39038))
for model in models:
    cands = read_torch(f'data/{dataset}/top64_cands/{model}.t7')
    labels = np.array(cands['labels'])
    l = l | (labels != -1)

In [22]:
# Zeshel
sum(l)

9110

In [25]:
# MedMentions
sum(l)

38065

In [64]:
dataset = 'medmentions'
data_len = 10000 if dataset == 'zeshel' else 39038
cands = {}
final_labels = [-1]*data_len
final_cands = {}
for model in models:
    cands[model] = read_torch(f'data/{dataset}/top64_cands/{model}.t7')
for i in range(len(cands[models[0]]['labels'])):
    correct_cand = None
    found = False
    cands_union = set()
    # Find correct candidate
    for model in cands:
        if not found and cands[model]['labels'][i] != -1:
            correct_cand = cands[model]['candidates'][i][cands[model]['labels'][i]]
            found = True
        cands_union = set().union(cands_union, cands[model]['candidates'][i])
    final_cands[i] = list(cands_union)
    if correct_cand is not None:
        final_labels[i] = final_cands[i].index(correct_cand)

# Find max. cand length
maxlen = -1
for i in range(len(final_cands)):
    maxlen = max(maxlen, len(final_cands[i]))
maxlen

# Normalize length of all candidates to be able to convert to numpy matrix
for i in final_cands:
    add_dummies = maxlen - len(final_cands[i])
    if add_dummies > 0:
        dummy_label_idx = final_labels[i] - 1
        if dummy_label_idx < 0:
            dummy_label_idx = final_labels[i] + 1
        assert 0 <= dummy_label_idx < len(final_cands[i])
        dummy_label = final_cands[i][dummy_label_idx]
        final_cands[i] += [dummy_label]*add_dummies
    assert len(final_cands[i]) == maxlen
        
torch.save({
    'mode': 'union',
    'candidates': final_cands,
    'labels': final_labels
}, f'data/{dataset}/top64_cands/union.t7')